In [ ]:
#Importing dependencies
import cv2 as cv
import mediapipe as mp
import time
import numpy as np
import random 


#Defining global variables
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
cap = cv.VideoCapture(0)
prev_time = 0
player_batting = True
player_sign = None
computer_sign = None
player_score = 0
computer_score = 0
innings = 0
toss = 0
cv.namedWindow('Hand Tracking', cv.WINDOW_NORMAL)


#Function to check if finger is raised
def finger_raised(finger):
    a,b,c = finger[0],finger[1],finger[2]
    a,b,c =np.array(a),np.array(b),np.array(c)
    angle = np.abs(np.degrees(np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])))
    if angle > 180:
        angle = 360 - angle
    if angle > 158:
        return True
    else:
        return False


#Function to detect the sign based on the landmarks  
def sign_detection(hand_landmarks_req):
    landmarks = [(lm.x,lm.y) for lm in hand_landmarks_req.landmark]
    index = [landmarks[5],landmarks[6],landmarks[7]]
    middle = [landmarks[9],landmarks[10],landmarks[11]]
    ring = [landmarks[13],landmarks[14],landmarks[15]]
    pinky = [landmarks[17],landmarks[18],landmarks[19]]               
    thumb = [landmarks[2],landmarks[3],landmarks[4]]

    if finger_raised(index) and (not(finger_raised(middle) or finger_raised(ring) or finger_raised(pinky) or finger_raised(thumb))):
        player_sign = 1
    elif finger_raised(index) and finger_raised(middle) and (not(finger_raised(ring) or finger_raised(pinky) or finger_raised(thumb))):
        player_sign = 2
    elif finger_raised(index) and finger_raised(middle) and finger_raised(ring) and (not(finger_raised(pinky) or finger_raised(thumb))):
        player_sign = 3
    elif finger_raised(index) and finger_raised(middle) and finger_raised(ring) and finger_raised(pinky) and (not(finger_raised(thumb))):
        player_sign = 4
    elif finger_raised(index) and finger_raised(middle) and finger_raised(ring) and finger_raised(pinky) and finger_raised(thumb):
        player_sign = 5
    elif finger_raised(thumb) and (not(finger_raised(index) or finger_raised(middle) or finger_raised(ring) or finger_raised(pinky))):
        player_sign = 6
    elif not(finger_raised(thumb) or finger_raised(index) or finger_raised(middle) or finger_raised(ring) or finger_raised(pinky)):
        player_sign = 10
    else:
        player_sign = None
    return player_sign

#Tossing
while toss not in [1,2]:
    toss = int(input("Enter 1 for Heads and 2 for Tails: "))
toss_result = random.choices([1,2],weights=[1,1],k=1)[0]
if toss == toss_result:
    time.sleep(1)
    print("You won the toss!")
    player_batting = True
else:
    time.sleep(1)
    print("You lost the toss!")
    player_batting = False


#Main Loop
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    print("Game Start!")
    time.sleep(1)
    print("You are Batting!")if player_batting else print("Computer is Batting!")
    
    #One iteration of this loop is one ball
    while innings != 2:
        computer_sign = random.choices([1,2,3,4,5,6,10],weights=[1,1,1,1,1.5,1.5,1.7],k=1)[0]    

        #Waiting for player to show sign
        count = 3
        start_time = time.time()
        prev_time = time.time()
        while count > -1:
            if time.time() - start_time > 1.5:
                if count != 0:
                    print(f"Show sign in {count}")
                count -= 1
                start_time = time.time()
            ret,frame = cap.read()
            if not ret:
                continue
            frame = cv.flip(frame,1)
            rgb_frame = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
            results = hands.process(rgb_frame)
            if not results.multi_hand_landmarks:
                continue
            elif results.multi_hand_landmarks:
                for hand_landmarks2 in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame,hand_landmarks2,mp_hands.HAND_CONNECTIONS)
            player_sign = sign_detection(hand_landmarks_req=hand_landmarks2)
            cv.putText(frame,f"Player Sign: {player_sign}",(10,30),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            cv.imshow('Hand Tracking',frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        
        #If player showed a valid sign
        ret,frame = cap.read()
        if not ret:
            continue
        frame = cv.flip(frame,1)
        rgb_frame = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        if not results.multi_hand_landmarks:
            continue
        elif results.multi_hand_landmarks:
            for hand_landmarks1 in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame,hand_landmarks1,mp_hands.HAND_CONNECTIONS)

            #Game Logic   
            player_sign = sign_detection(hand_landmarks_req=hand_landmarks1)
            cv.putText(frame,f"Player sign: {player_sign}",(10,30),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            cv.imshow('Hand Tracking',frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                    break
            print(f"Player sign: {player_sign}")
            print(f"Computer sign: {computer_sign}")
            if player_sign == None:
                print("Invalid sign!")
                continue

            #player batting logic
            elif player_batting == True:
                if player_sign == computer_sign:
                    player_batting = False
                    innings += 1
                    print(f"Out!, Your Score is: {player_score}")
                    time.sleep(1.5)
                else:
                    player_score += player_sign
                    print(f"Your score: {player_score}")
                    
            #Player bowling logic
            elif player_batting == False:
                if player_sign == computer_sign:
                    player_batting = True
                    innings += 1
                    print(f"Out!, Computer's Score is: {computer_score}")
                    time.sleep(1.5)
                else:
                    computer_score += computer_sign
                    print(f"Computer's score: {computer_score}")

        
#After 2 innings
print("Game Over!")
print(f"Your score: {player_score}")
print(f"Computer's score: {computer_score}")
if player_score > computer_score:
    print("You win!")
elif player_score < computer_score:
    print("Computer wins!")
else:
    print("Match Draw!")
           
cap.release()
cv.destroyAllWindows()

You won the toss!
Game Start!
You are Batting!
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 5
Computer sign: 10
Your score: 5
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 10
Computer sign: 6
Your score: 15
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 3
Computer sign: 3
Out!, Your Score is: 15
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 4
Computer sign: 5
Computer's score: 5
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 1
Computer sign: 5
Computer's score: 10
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 4
Computer sign: 1
Computer's score: 11
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 5
Computer sign: 4
Computer's score: 15
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 6
Computer sign: 10
Computer's score: 25
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 6
Computer sign: 4
Computer's score: 29
Show sign in 3
Show sign in 2
Show sign in 1
Player sign: 5
Computer sign: 6
Co